In [1]:
!pip install pickle5

In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle5 as pickle
import gensim
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from csv import reader
from scipy import spatial
import functools
import shutil

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **READ & PRE-PROCESS PARTY TIME DF**

In [18]:
%%time
with open('/kaggle/input/lemmatizedpartytimedf/LemmatizedPartyTimeDF (1)','rb') as f:
    lemPartyTimeDf = pickle.load(f)
lemPartyTimeDf.reset_index(inplace=True)
lemPartyTimeDf.drop('index',axis=1,inplace=True)
lemPartyTimeDf

In [19]:
%%time
# -------------CHANGE UK AND THE INDEPENDENT GROUP FOR CHANGE ARE THE SAME PARTY, HENCE COMBINING THEIR DATA-------------
lemPartyTimeDf[(lemPartyTimeDf['party']=='The Independent Group for Change') | (lemPartyTimeDf['party']=='Change UK - The Independent Group')]
targetIndex = lemPartyTimeDf[lemPartyTimeDf['party']=='Change UK - The Independent Group'].index.values[0]
joineeIndex = lemPartyTimeDf[lemPartyTimeDf['party']=='The Independent Group for Change'].index.values[0]

print(len(lemPartyTimeDf.at[targetIndex,'sentence']), len(lemPartyTimeDf.at[joineeIndex,'sentence']))

lemPartyTimeDf.at[targetIndex,'sentence']+= lemPartyTimeDf.at[joineeIndex, 'sentence']
len(lemPartyTimeDf.at[targetIndex,'sentence'])

print(len(lemPartyTimeDf.at[targetIndex,'lemmatized']), len(lemPartyTimeDf.at[joineeIndex,'lemmatized']))
lemPartyTimeDf.at[targetIndex,'lemmatized']+= lemPartyTimeDf.at[joineeIndex,'lemmatized']
len(lemPartyTimeDf.at[targetIndex,'lemmatized'])

lemPartyTimeDf.at[targetIndex,'LengthLemmas']=len(lemPartyTimeDf.at[targetIndex,'lemmatized'])

lemPartyTimeDf.drop(joineeIndex,inplace=True)
lemPartyTimeDf.reset_index(inplace=True)
lemPartyTimeDf

In [20]:
lemPartyTimeDf.shape

In [75]:
%%time
lemPartyTimeDf['lemmas_delist'] = [','.join(map(str, l)) for l in lemPartyTimeDf['lemmatized']]
lemPartyTimeDf = lemPartyTimeDf[lemPartyTimeDf['lemmas_delist'].str.contains(r'\bbrexit\b')]
lemPartyTimeDf = lemPartyTimeDf[lemPartyTimeDf['lemmas_delist'].str.contains(r'\bleave\b')]
lemPartyTimeDf.shape


# **MODELS' CREATION & ALIGNMENT**

In [80]:
lemPartyTimeDf.reset_index(inplace=True)
lemPartyTimeDf

In [82]:
%%time

#  - x - x - x - CREATING & SAVING WORD2VEC MODELS FOR THE 680 MODELS - x - x - x 

dictOfModels = {}
'''#shutil.rmtree('./models-by-party-and-time')
os.makedirs('./models-by-party-and-time')
models_folder = './models-by-party-and-time'
'''
count = 1

for ind in range(0,lemPartyTimeDf.shape[0]): 
    tempDf = lemPartyTimeDf.loc[[ind],:]
    print('Creating Model no.', ind+1, ' ', lemPartyTimeDf.at[ind,'df_name'])
    #model = gensim.models.Word2Vec(lemPartyTimeDf.at[ind,'lemmatized'], min_count=1, vector_size=300, window = 5, sg = 1)
    model = gensim.models.Word2Vec(tempDf['lemmatized'], min_count=1, vector_size=300, window = 5, sg = 1)

    # Also saving model in a dict and exporting
    modelName ='model_'+ lemPartyTimeDf.at[ind,'df_name']
    #model.save(os.path.join(models_folder, modelName))

    dictOfModels[lemPartyTimeDf.at[ind,'df_name']] = model

In [83]:
len(dictOfModels.keys())

In [84]:
dictOfModels['df_t2_Social Democratic & Labour Party'].wv.index_to_key

In [85]:
lemPartyTimeDf.sort_values(by='LengthLemmas')

In [ ]:
'''%%time

# Load 30 Word2Vec models of MPs in T1 & T2

dictOfModels = {}
folderPath = '/kaggle/input/partymodelsfiles'
for file in os.listdir(folderPath):
    filePath = folderPath + '/' + file
    print(filePath)
    if(len(filePath.split('.'))>1):
        continue
    else:
        model = gensim.models.Word2Vec.load(filePath)
        dictOfModels[file] = model'''

In [86]:
def intersection_align_gensim(m1, m2, words=None):
    """
    Intersect two gensim word2vec models, m1 and m2.
    Only the shared vocabulary between them is kept.
    If 'words' is set (as list or set), then the vocabulary is intersected with this list as well.
    Indices are re-organized from 0..N in order of descending frequency (=sum of counts from both m1 and m2).
    These indices correspond to the new syn0 and syn0norm objects in both gensim models:
        -- so that Row 0 of m1.syn0 will be for the same word as Row 0 of m2.syn0
        -- you can find the index of any word on the .index2word list: model.index2word.index(word) => 2
    The .vocab dictionary is also updated for each model, preserving the count but updating the index.
    """
    print('Vocab function')
    # Get the vocab for each model
    vocab_m1 = set(m1.wv.index_to_key)
    vocab_m2 = set(m2.wv.index_to_key)

    # Find the common vocabulary
    common_vocab = vocab_m1 & vocab_m2
    if words: common_vocab &= set(words)
    print('vocabs',list(common_vocab)[0], list(vocab_m1)[0], list(vocab_m2)[0])
    print('1. Common vocab length', len(common_vocab))

    # If no alignment necessary because vocab is identical...
    if not vocab_m1 - common_vocab and not vocab_m2 - common_vocab:
        return (m1,m2)

    # Otherwise sort by frequency (summed for both)
    common_vocab = list(common_vocab)
    common_vocab.sort(key=lambda w: m1.wv.get_vecattr(w, "count") + m2.wv.get_vecattr(w, "count"), reverse=True)
    # print(len(common_vocab))
    print('2. Common vocab length', len(common_vocab))
    # Then for each model...
    for m in [m1, m2]:
        # Replace old syn0norm array with new one (with common vocab)
        indices = [m.wv.key_to_index[w] for w in common_vocab]
        old_arr = m.wv.vectors
        new_arr = np.array([old_arr[index] for index in indices])
        m.wv.vectors = new_arr

        # Replace old vocab dictionary with new one (with common vocab)
        # and old index2word with new one
        new_key_to_index = {}
        new_index_to_key = []
        for new_index, key in enumerate(common_vocab):
            new_key_to_index[key] = new_index
            new_index_to_key.append(key)
        m.wv.key_to_index = new_key_to_index
        m.wv.index_to_key = new_index_to_key
        
        print(len(m.wv.key_to_index), len(m.wv.vectors))
    print('Vocab function returning models with shapes', m1.wv.vectors.shape, m2.wv.vectors.shape)    
    return (m1,m2)

In [87]:
# Function to align two spaces with orthogunal procrustes
def smart_procrustes_align_gensim(base_embed, other_embed, words=None):
    print('shapes', base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    """
    Original script: https://gist.github.com/quadrismegistus/09a93e219a6ffc4f216fb85235535faf
    Procrustes align two gensim word2vec models (to allow for comparison between same word across models).
    Code ported from HistWords <https://github.com/williamleif/histwords> by William Hamilton <wleif@stanford.edu>.
        
    First, intersect the vocabularies (see `intersection_align_gensim` documentation).
    Then do the alignment on the other_embed model.
    Replace the other_embed model's syn0 and syn0norm numpy matrices with the aligned version.
    Return other_embed.
    If `words` is set, intersect the two models' vocabulary with the vocabulary in words (see `intersection_align_gensim` documentation).
    """

    # make sure vocabulary and indices are aligned
    in_base_embed, in_other_embed = intersection_align_gensim(base_embed, other_embed, words=words)
    print('a, the shapes received are', in_base_embed.wv.vectors.shape, other_embed.wv.vectors.shape)
    print(str(len(list(in_base_embed.wv.index_to_key))))
    print(str(len(list(in_other_embed.wv.index_to_key))))

    # re-filling the normed vectors
    in_base_embed.wv.fill_norms(force=True)
    in_other_embed.wv.fill_norms(force=True)
    
    # get the (normalized) embedding matrices
    base_vecs = in_base_embed.wv.get_normed_vectors()
    
    other_vecs = in_other_embed.wv.get_normed_vectors()
    # just a matrix dot product with numpy
    m = other_vecs.T.dot(base_vecs) 

    # SVD method from numpy
    u, _, v = np.linalg.svd(m)

    # another matrix operation
    ortho = u.dot(v) 

    # Replace original array with modified one, i.e. multiplying the embedding matrix by "ortho"
    other_embed.wv.vectors = (other_embed.wv.vectors).dot(ortho)    
    print('Procrustes function returning')
    return other_embed

In [88]:
%%time
# Reduces to a vocab of 1121 words

modelsToAlign = list(dictOfModels.values())
for i in range(0,len(modelsToAlign)-1):
    functools.reduce(smart_procrustes_align_gensim, modelsToAlign)

In [ ]:
'''# Alignment check - compare vocab sizes

for ind in range(0,len(dictOfModels.values())-1):
    if(len(dictOfModels[listDfsKeep[ind]].wv.index_to_key)!=len(dictOfModels[listDfsKeep[ind+1]].wv.index_to_key)):
        print('Vocabs not similar')
        
print('Vocab Size', len(dictOfModels[listDfsKeep[ind]].wv.index_to_key))

# Common vocab size 197 without filtering for any words of interest
# Common vocab size 245 after lemmatization and only considering common words - brexit, leave, '''

In [89]:
len(dictOfModels['df_t2_Social Democratic & Labour Party'].wv.index_to_key)

In [91]:
len(dictOfModels['df_t1_Green Party'].wv.index_to_key)

In [92]:
%%time
shutil.rmtree('./models-by-party-and-time')
os.makedirs('./models-by-party-and-time')
models_folder = './models-by-party-and-time'


# Exporting aligned models
count=1
for k in dictOfModels.keys(): 
    print('Saving Model no.', count, ' ', k)
    model = dictOfModels[k]
    
    modelName ='model_'+ k
    model.save(os.path.join(models_folder, modelName))
    count=count+1

In [93]:
%%time
!zip -r file.zip /kaggle/working/models-by-party-and-time

In [ ]:
'''change = ['exiting', 'seaborne', 'eurotunnel', 'withdrawal', 'departures', 'unicorn', 'remainers', 'exit', 'surrender',
          'departure', 'triggering', 'stockpiling', 'expulsion', 'blindfold', 'cliff', 'lighter', 'exits', 'triggered',
          'brexiteer', 'soft', 'plus', 'trigger', 'backroom', 'invoked', 'protesting', 'brexit', 'edge', 'canary', 
          'unicorns', 'withdrawing', 'invoking', 'withdrawn', 'manor', 'brexiteers', 'fanatics', 'postponement', 
          'currencies', 'currency', 'operability', 'operable', 'leavers', 'invoke', 'article', 'eurozone', 'clueless',
          'surrendered', 'cake', 'red', 'euroscepticism', 'prorogation', 'lining', 'gove', 'norway', 'deflationary',
          'moribund', 'eurosceptic', 'deutschmark', 'courting', 'deal', 'withdraw', 'dab', 'withdrawals', 'eurosceptics',
          'surrendering', 'aldous', 'lanarkshire', 'leaving', 'signifying', 'roofs', 'ceded', 'absentia', 'treachery',
          'dollar', 'canada', 'pragmatist', 'oven', 'ready', 'brexiters', 'control', 'capitulation', 'leave', 'referendum',
          'agreement', 'prorogue', 'smoothest', 'depreciate', 'managed', 'mutiny', 'overvalued', 'ideologues', 'foreign',
          'eec', 'war', 'prorogued', 'hannan', 'appease', 'pendolino', 'southbound', 'left', 'line', 'hard', 'bill']

no_change = ['prime', 'even', 'parliament', 'mr', 'eu', 'bill', 'future', 'care', 'well', 'constituency', 'tax',
             'children', 'uk', 'business', 'european', 'report', 'case', 'sure', 'like', 'see', 'state', 'health',
             'order', 'back', 'new', 'hope', 'local', 'national', 'country', 'secretary', 'public', 'right', 'much',
             'say', 'first', 'minister', 'labour', 'look', 'system', 'whether', 'members', 'million', 'good', 'today', 
             'services', 'clear', 'help', 'time', 'place', 'put', 'last', 'must', 'money', 'one', 'way', 'friend',
             'work', 'would', 'think', 'two', 'great', 'could', 'lady', 'us', 'come', 'however', 'may', 'going', 'go',
             'given', 'need', 'year', 'debate', 'might', 'part', 'get', 'want', 'make', 'point', 'committee', 'years',
             'also', 'know', 'government', 'take', 'house', 'agree', 'member', 'number', 'across', 'made', 'give', 
             'gentleman', 'important', 'said', 'people', 'issue', 'support', 'ensure']'''

In [ ]:
'''# Slightly modified to now find the cosine difference between provided vectors instead of
# fetching vectors from known models 
def cosine_similarity(vec1, vec2):
  sc = 1-spatial.distance.cosine(vec1, vec2)
  return sc'''

In [ ]:
'''%%time
cosine_similarity_df = pd.DataFrame(columns = ('Word', 'Cosine_similarity'))'''

In [ ]:
'''change_words = ['war','leaving','brexit','red','agreement','control','hard','managed',
                'article','foreign','leave','deal','line','left','bill']

#dictOfModels['df_t2_1423'].wv.index_to_key
#dictOfModels['df_t2_1423'].layer1_size
words_of_interest = change+no_change
for word in words_of_interest:
        # Compute average in T1 and average in T2 and store average vectors   
        
kis = ['df_t2_1423','df_t2_3945', 'df_t2_4070']
modelsSum = np.zeros(dictOfModels['df_t2_1423'].layer1_size)

for k in kis:
    #Something like this to refer to t's vector
    #vector = dictOfModels[k].wv.get_index(t)#
    vector = dictOfModels[k].wv[t]
    modelsSum = np.add(modelsSum, vector)
    
avgEmbedding =np.divide(modelsSum, len(kis))
'''

In [ ]:
#cosine_similarity_df_sorted = cosine_similarity_df.sort_values(by='Cosine_similarity', ascending=True)